In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import os
import timm
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

# ✅ BEiT 분류 모델 정의
class BEiTClassifier(nn.Module):
    def __init__(self):
        super(BEiTClassifier, self).__init__()
        self.backbone = timm.create_model(
            'beit_base_patch16_224',
            pretrained=True,
            num_classes=3  # 클래스 수
        )

    def forward(self, x):
        return self.backbone(x)

# ✅ 사용자 정의 Dataset
class ScrapClassificationDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None, label_encoder=None):
        self.data = dataframe.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform
        self.label_encoder = label_encoder or LabelEncoder()
        self.data['class_idx'] = self.label_encoder.fit_transform(self.data['weight_class'])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.data.iloc[idx]['filename'])
        image = Image.open(img_path).convert('RGB')
        label = torch.tensor(self.data.iloc[idx]['class_idx'], dtype=torch.long)
        if self.transform:
            image = self.transform(image)
        return image, label

# ✅ 평가 함수
def evaluate_classification_metrics(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    acc = correct / total * 100
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')
    return acc, precision, recall, f1

# ✅ 경로 설정
csv_path = r"C:\Users\pyw20\OneDrive\바탕 화면\work\train.csv"
img_dir = r"C:\Users\pyw20\OneDrive\바탕 화면\work\train_images"

# ✅ 데이터 로드 및 전처리
df = pd.read_csv(csv_path)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # BEiT는 224 입력
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# ✅ 5-Fold 학습
kf = KFold(n_splits=5, shuffle=True, random_state=42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
fold_results = []

for fold, (train_idx, val_idx) in enumerate(kf.split(df)):
    print(f"\n📂 Fold {fold+1}")

    train_df = df.iloc[train_idx]
    val_df = df.iloc[val_idx]
    label_encoder = LabelEncoder()
    label_encoder.fit(df['weight_class'])

    train_dataset = ScrapClassificationDataset(train_df, img_dir, transform, label_encoder)
    val_dataset = ScrapClassificationDataset(val_df, img_dir, transform, label_encoder)
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

    model = BEiTClassifier().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=1e-4)

    # 학습 (10 epoch)
    for epoch in range(10):
        model.train()
        total_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"  Epoch {epoch+1} - Loss: {total_loss / len(train_loader):.4f}")

    # 평가
    acc, precision, recall, f1 = evaluate_classification_metrics(model, val_loader, device)
    fold_results.append((acc, precision, recall, f1))
    print(f"✅ Fold {fold+1} Accuracy: {acc:.2f}%")
    print(f"📍 Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

# ✅ 최종 평균 출력
accs, precs, recalls, f1s = zip(*fold_results)
print("\n📊 최종 평균 성능:")
print(f"🔹 Accuracy: {np.mean(accs):.2f}%")
print(f"🔹 Precision: {np.mean(precs):.4f}")
print(f"🔹 Recall:    {np.mean(recalls):.4f}")
print(f"🔹 F1 Score:  {np.mean(f1s):.4f}")



📂 Fold 1


model.safetensors:   0%|          | 0.00/350M [00:00<?, ?B/s]

  Epoch 1 - Loss: 1.0899
  Epoch 2 - Loss: 1.0127
  Epoch 3 - Loss: 0.8578
  Epoch 4 - Loss: 0.7986
  Epoch 5 - Loss: 0.4580
  Epoch 6 - Loss: 0.1987
  Epoch 7 - Loss: 0.2505
  Epoch 8 - Loss: 0.0825
  Epoch 9 - Loss: 0.0971
  Epoch 10 - Loss: 0.0407


c:\Users\pyw20\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


✅ Fold 1 Accuracy: 52.38%
📍 Precision: 0.3667, Recall: 0.5333, F1 Score: 0.4222

📂 Fold 2
  Epoch 1 - Loss: 1.0713
  Epoch 2 - Loss: 1.0443
  Epoch 3 - Loss: 1.0212
  Epoch 4 - Loss: 0.8798
  Epoch 5 - Loss: 0.6377
  Epoch 6 - Loss: 0.4304
  Epoch 7 - Loss: 0.1965
  Epoch 8 - Loss: 0.3208
  Epoch 9 - Loss: 0.1660
  Epoch 10 - Loss: 0.0885
✅ Fold 2 Accuracy: 40.00%
📍 Precision: 0.4651, Recall: 0.5606, F1 Score: 0.4202

📂 Fold 3
  Epoch 1 - Loss: 1.1005
  Epoch 2 - Loss: 1.0011
  Epoch 3 - Loss: 0.8725
  Epoch 4 - Loss: 0.7112
  Epoch 5 - Loss: 0.6934
  Epoch 6 - Loss: 0.4805
  Epoch 7 - Loss: 0.4285
  Epoch 8 - Loss: 0.3789
  Epoch 9 - Loss: 0.2351
  Epoch 10 - Loss: 0.1786
✅ Fold 3 Accuracy: 50.00%
📍 Precision: 0.4095, Recall: 0.5524, F1 Score: 0.4571

📂 Fold 4
  Epoch 1 - Loss: 1.1088
  Epoch 2 - Loss: 1.0759
  Epoch 3 - Loss: 0.9649
  Epoch 4 - Loss: 0.6149
  Epoch 5 - Loss: 0.3830
  Epoch 6 - Loss: 0.2130
  Epoch 7 - Loss: 0.1295
  Epoch 8 - Loss: 0.0481
  Epoch 9 - Loss: 0.0287
  E

c:\Users\pyw20\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
